Imports

In [2]:
# !pip install scipy==1.6.0
# !pip install matplotlib==3.1.0

In [1]:
import os, json, random
random.seed(1)
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display, Image
from tqdm import tqdm
from collections import OrderedDict
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# AIF360
import aif360
from aif360.datasets import CompasDataset
from aif360.sklearn.datasets import fetch_compas
# fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, DatasetMetric
from aif360.metrics.common_utils import compute_metrics
from aif360.metrics.utils import compute_num_instances
# data preprocessing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas
# explainers
from aif360.explainers import MetricTextExplainer
# bias mitigation techniques
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing, PrejudiceRemover, GerryFairClassifier
from aif360.sklearn.inprocessing import AdversarialDebiasing as SKLearnAdversarialDebiasing
from aif360.algorithms.inprocessing.gerryfair.clean import array_to_tuple
from aif360.algorithms.inprocessing.gerryfair.auditor import Auditor
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, RejectOptionClassification
from aif360.sklearn.utils import check_inputs, check_groups

In [3]:
# TensorFlow
import tensorflow
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [4]:
import sklearn
# scalers
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree, linear_model
# metrics
from sklearn.metrics import accuracy_score, roc_curve, classification_report, confusion_matrix
# kernels
from sklearn.kernel_ridge import KernelRidge

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.datasets import make_blobs
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.inspection import permutation_importance

Helper Functions

In [5]:
FILES = '/Users/megantennies/FYP/saved data'

In [6]:
def save_to_json(filename, data):
    with open(os.path.join(FILES, filename), 'w') as write:
        json.dump(data, write)

In [7]:
def json_to_df(filename):
    with open(filename, 'r') as load:
        data = json.load(load)
    df = pd.DataFrame(data)
    return df

In [8]:
def df_to_json(filename, df):
    df.to_json(os.path.join(FILES, filename), orient = 'split', 
    compression = 'infer', index = True)

In [9]:
def plot_metric_graphs(metric_name, filename):
    pass

Data

In [10]:
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]
original_dataset = load_preproc_data_compas(['race'])

In [11]:
default_mappings = {'label_maps': [{1.0: 'Recid', 0.0: 'Non-Recid'}], 
    'protected_attribute_maps': [{1.0: 'Male', 0.0: 'Female'}, 
    {1.0: 'White', 0.0: 'Non-White'}]}
metrics = ['Statistical parity difference', 'Average odds difference', 
    'Equal opportunity difference']

In [12]:
original_train, original_val_test = original_dataset.split([0.7], shuffle = True)
original_val, original_test = original_val_test.split([0.5], shuffle = True)

In [13]:
def describe(train = None, val = None, test = None):
    if train is not None:
        display(Markdown('#### Training dataset shape'))
        print(train.features.shape)
    if val is not None:
        display(Markdown('#### Validation dataset shape'))
        print(val.features.shape)
    display(Markdown('#### Test dataset shape'))
    print(test.features.shape)
    display(Markdown('#### Favorable and unfavorable labels'))
    print(test.favorable_label, test.unfavorable_label)
    display(Markdown('#### Protected attribute names'))
    print(test.protected_attribute_names)
    display(Markdown('#### Privileged and unprivileged protected attribute values'))
    print(test.privileged_protected_attributes, test.unprivileged_protected_attributes)
    display(Markdown("#### Dataset feature names"))
    print(train.feature_names)

In [14]:
describe(original_train, original_val, original_test)

#### Training dataset shape

(3694, 10)


#### Validation dataset shape

(792, 10)


#### Test dataset shape

...

['race']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['sex', 'race', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'priors_count=0', 'priors_count=1 to 3', 'priors_count=More than 3', 'c_charge_degree=F', 'c_charge_degree=M']


In [15]:
original_metric = BinaryLabelDatasetMetric(original_train, 
    unprivileged_groups = unprivileged_groups, 
    privileged_groups = privileged_groups)

In [16]:
original_explainer = MetricTextExplainer(original_metric)
display(Markdown('#### Original COMPAS training data'))

original_train_metric = BinaryLabelDatasetMetric(original_train, 
    unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)
print('Training data: Difference in mean outcomes between unprivileged and privileged groups = %f' % original_train_metric.mean_difference())

original_val_metric = BinaryLabelDatasetMetric(original_val, 
    unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)
print('Validation data: Difference in mean outcomes between unprivileged and privileged groups = %f' % original_val_metric.mean_difference())

original_test_metric = BinaryLabelDatasetMetric(original_test, 
    unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)
print('Testing data: Difference in mean outcomes between unprivileged and privileged groups = %f' % original_test_metric.mean_difference())

#### Original COMPAS training data

Training data: Difference in mean outcomes between unprivileged and privileged groups = -0.121970
Validation data: Difference in mean outcomes between unprivileged and privileged groups = -0.131346
Testing data: Difference in mean outcomes between unprivileged and privileged groups = -0.180519


LR

In [17]:
original_scaler = StandardScaler()
X_train = original_scaler.fit_transform(original_train.features)
y_train = original_train.labels.ravel()
w_train = original_train.instance_weights.ravel()

In [18]:
lr = LogisticRegression()
lr.fit(X_train, y_train, sample_weight = original_train.instance_weights)
y_train_preds = lr.predict(X_train)

In [19]:
pos_ind = np.where(lr.classes_ == original_train.favorable_label)[0][0]

In [20]:
original_train_preds = original_train.copy()
original_train_preds.labels = y_train_preds

original_val_preds = original_val.copy(deepcopy = True)
X_val = original_scaler.transform(original_val_preds.features)
y_val = original_val_preds.labels
original_val_preds.scores = lr.predict_proba(X_val)[:, pos_ind].reshape(-1, 1)

original_test_preds = original_test.copy(deepcopy = True)
X_test = original_scaler.transform(original_test_preds.features)
y_test = original_test_preds.labels
original_test_preds.scores = lr.predict_proba(X_test)[:, pos_ind].reshape(-1, 1)

In [21]:
num_thresh = 100
bal_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)

for idx, class_thresh in enumerate(class_thresh_arr):
    fav_inds = original_val_preds.scores > class_thresh
    original_val_preds.labels[fav_inds] = original_val_preds.favorable_label
    original_val_preds.labels[~fav_inds] = original_val_preds.unfavorable_label

    original_val_metric = ClassificationMetric(original_val, 
        original_val_preds, unprivileged_groups = unprivileged_groups, 
        privileged_groups = privileged_groups)

    bal_arr[idx] = 0.5 * (original_val_metric.true_positive_rate() 
        + original_val_metric.true_negative_rate())

In [22]:
best_ind = np.where(bal_arr == np.max(bal_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

display(Markdown('#### Original COMPAS data'))
print('Best balanced accuracy (no transforming): %.4f' % np.max(bal_arr))
print('Optimal classification threshold (no transforming): %.4f' % best_class_thresh)

#### Original COMPAS data

Best balanced accuracy (no transforming): 0.6720
Optimal classification threshold (no transforming): 0.5049


In [34]:
bal_acc_arr = []
disp_imp_arr = []
avg_odds_diff_arr = []
eq_opp_diff_arr = []
outcome_unfair_arr = []
acc_equal_arr = []

In [35]:
display(Markdown('#### Predictions from the original testing data'))
print('Classification threshold used: %.4f' % best_class_thresh)

for thresh in tqdm(class_thresh_arr):
    if thresh == best_class_thresh:
        disp = True
    else:
        disp = False
    fav_inds = original_test_preds.scores > thresh
    original_test_preds.labels[fav_inds] = original_test_preds.favorable_label
    original_test_preds.labels[~fav_inds] = original_test_preds.unfavorable_label
    
    metric_test = compute_metrics(original_test, original_test_preds, \
        unprivileged_groups, privileged_groups, disp = disp)
    class_metric_test = ClassificationMetric(original_test, original_test_preds, \
        unprivileged_groups, privileged_groups)
    
    bal_acc_arr.append(metric_test['Balanced accuracy'])
    avg_odds_diff_arr.append(metric_test['Average odds difference'])
    disp_imp_arr.append(metric_test['Disparate impact'])
    eq_opp_diff_arr.append(metric_test['Equal opportunity difference'])
    outcome_unfair_arr.append(class_metric_test.false_discovery_rate_difference() \
        + class_metric_test.false_positive_rate_difference())
    acc_equal_arr.append((class_metric_test.true_positive_rate(privileged = False) + \
        class_metric_test.true_negative_rate(privileged = False)) - \
            (class_metric_test.true_positive_rate(privileged = True) + \
                class_metric_test.true_negative_rate(privileged = True)))

#### Predictions from the original testing data

 69%|██████▉   | 69/100 [00:00<00:00, 315.73it/s]

Classification threshold used: 0.4258
Balanced accuracy = 0.6605
Statistical parity difference = -0.2801
Disparate impact = 0.6556
Average odds difference = -0.2425
Equal opportunity difference = -0.2173
Theil index = 0.1652


invalid value encountered in double_scalars
100%|██████████| 100/100 [00:00<00:00, 361.95it/s]


In [39]:
LR_bal_acc = np.interp(best_class_thresh, class_thresh_arr, bal_acc_arr)
LR_acc_equal = np.interp(best_class_thresh, class_thresh_arr, acc_equal_arr)
LR_disp_imp = np.interp(best_class_thresh, class_thresh_arr, disp_imp_arr)
LR_out_unf = np.interp(best_class_thresh, class_thresh_arr, outcome_unfair_arr)
LR_avg_odds = np.interp(best_class_thresh, class_thresh_arr, avg_odds_diff_arr)
LR_eq_odds = np.interp(best_class_thresh, class_thresh_arr, eq_opp_diff_arr)

In [40]:
LR_results = {'Metric': ['Balanced Accuracy', 'Accuracy Equality', 'Disparate Impact', 'Outcome Unfairess', 'Average Odds Difference', 'Equal Opportunity Difference'],
    'Logistic Regression': [LR_bal_acc, LR_acc_equal, LR_disp_imp, LR_out_unf, LR_avg_odds, LR_eq_odds]}
LR_results_df = pd.DataFrame(LR_results)
LR_results_df

,Metric,Logistic Regression
0,Balanced Accuracy,0.660480
1,Accuracy Equality,0.050261
2,Disparate Impact,0.655639
3,Outcome Unfairess,-0.196279
4,Average Odds Difference,-0.242456
5,Equal Opportunity Difference,-0.217325


In [46]:
df_to_json(filename = 'LR_results.json', df = LR_results_df)

CEOD

In [29]:
def get_cost_constraint(constraints, dataset):
    validation_constraints = {'Cost Constraint': [], 'Difference in GFPR': [], 'Difference in GFNR': []}
    testing_constraints = {'Cost Constraint': [], 'Difference in GFPR': [], 'Difference in GFNR': []}
    if dataset == 'Validation':
        for constraint in constraints:
            ceo = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups, unprivileged_groups = unprivileged_groups, 
            cost_constraint = constraint, seed = 12345679)
            ceo.fit(original_val, original_val_preds)
            transf_val_preds = ceo.predict(original_val_preds)
            post_val_metric = ClassificationMetric(original_val_preds, transf_val_preds, unprivileged_groups = unprivileged_groups, 
                privileged_groups = privileged_groups)
            validation_constraints['Cost Constraint'].append(constraint)
            validation_constraints['Difference in GFPR'].append(post_val_metric.difference(post_val_metric.generalized_false_positive_rate))
            validation_constraints['Difference in GFNR'].append(post_val_metric.difference(post_val_metric.generalized_false_negative_rate))
        return pd.DataFrame(validation_constraints)
    elif dataset == 'Testing':
        for constraint in constraints:
            ceo = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups, unprivileged_groups = unprivileged_groups, 
            cost_constraint = constraint, seed = 12345679)
            ceo.fit(original_test, original_test_preds)
            transf_test_preds = ceo.predict(original_test_preds)
            post_test_metric = ClassificationMetric(original_test_preds, transf_test_preds, unprivileged_groups = unprivileged_groups, 
                privileged_groups = privileged_groups)
            testing_constraints['Cost Constraint'].append(constraint)
            testing_constraints['Difference in GFPR'].append(post_test_metric.difference(post_test_metric.generalized_false_positive_rate))
            testing_constraints['Difference in GFNR'].append(post_test_metric.difference(post_test_metric.generalized_false_negative_rate))       
        return pd.DataFrame(testing_constraints)

In [30]:
constraints = ['fpr', 'fnr', 'weighted']
get_cost_constraint(constraints = constraints, dataset = 'Validation')

invalid value encountered in double_scalars
invalid value encountered in double_scalars


,Cost Constraint,Difference in GFPR,Difference in GFNR
0,fpr,-0.148025,NaN
1,fnr,-0.113512,NaN
2,weighted,-0.127954,NaN


In [28]:
get_cost_constraint(constraints = constraints, dataset = 'Testing')

,Cost Constraint,Difference in GFPR,Difference in GFNR
0,fpr,-0.070120,0.194528
1,fnr,-0.214103,0.083026
2,weighted,-0.143638,0.094765


In [32]:
ceo = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups, 
    unprivileged_groups = unprivileged_groups, cost_constraint = 'weighted')
ceo.fit(original_train, original_train)

In [36]:
transf_val_preds = ceo.predict(original_val_preds)
transf_test_preds = ceo.predict(original_test_preds)

In [37]:
transf_val_metric = ClassificationMetric(original_val_preds, transf_val_preds, unprivileged_groups = unprivileged_groups, 
    privileged_groups = privileged_groups)
transf_test_metric = ClassificationMetric(original_test_preds, transf_test_preds, unprivileged_groups = unprivileged_groups, 
    privileged_groups = privileged_groups)

In [40]:
pre_transf_val_metric = ClassificationMetric(original_val, original_val_preds, unprivileged_groups = unprivileged_groups, 
    privileged_groups = privileged_groups)
pre_transf_test_metric = ClassificationMetric(original_test, original_test_preds, unprivileged_groups = unprivileged_groups, 
    privileged_groups = privileged_groups)

In [43]:
# assert np.abs(transf_val_metric.difference(transf_val_metric.generalized_false_negative_rate)) \
#     < np.abs(pre_transf_val_metric.difference(pre_transf_val_metric.generalized_false_negative_rate)) 

In [44]:
# assert np.abs(transf_test_metric.difference(transf_test_metric.generalized_false_negative_rate)) \
#     < np.abs(pre_transf_test_metric.difference(pre_transf_test_metric.generalized_false_negative_rate)) 

In [45]:
CEOD_bal_acc_arr = []
CEOD_disp_imp_arr = []
CEOD_avg_odds_diff_arr = []
CEOD_eq_opp_diff_arr = []
CEOD_outcome_unfair_arr = []
CEOD_acc_equal_arr = []

In [46]:
display(Markdown('#### Predictions from the original testing data'))
print('Classification threshold used: %.4f' % best_class_thresh)

for thresh in tqdm(class_thresh_arr):
    if thresh == best_class_thresh:
        disp = True
    else:
        disp = False

    fav_inds = transf_test_preds.scores > thresh
    transf_test_preds.labels[fav_inds] = transf_test_preds.favorable_label
    transf_test_preds.labels[~fav_inds] = transf_test_preds.unfavorable_label
    
    ceod_metric_test = compute_metrics(original_test, transf_test_preds, \
        unprivileged_groups, privileged_groups, disp = disp)
    ceod_class_metric_test = ClassificationMetric(original_test, transf_test_preds, \
        unprivileged_groups, privileged_groups)
    
    CEOD_bal_acc_arr.append(ceod_metric_test['Balanced accuracy'])
    CEOD_avg_odds_diff_arr.append(ceod_metric_test['Average odds difference'])
    CEOD_disp_imp_arr.append(ceod_metric_test['Disparate impact'])
    CEOD_eq_opp_diff_arr.append(ceod_metric_test['Equal opportunity difference'])
    CEOD_outcome_unfair_arr.append(ceod_class_metric_test.false_discovery_rate_difference() \
        + ceod_class_metric_test.false_positive_rate_difference())
    CEOD_acc_equal_arr.append((ceod_class_metric_test.true_positive_rate(privileged = False) + \
        ceod_class_metric_test.true_negative_rate(privileged = False)) - \
            (ceod_class_metric_test.true_positive_rate(privileged = True) + \
                ceod_class_metric_test.true_negative_rate(privileged = True)))

#### Predictions from the original testing data

 32%|███▏      | 32/100 [00:00<00:00, 317.60it/s]

Classification threshold used: 0.5049
Balanced accuracy = 0.6743
Statistical parity difference = -0.3044
Disparate impact = 0.5916
Average odds difference = -0.2526
Equal opportunity difference = -0.2303
Theil index = 0.2091


100%|██████████| 100/100 [00:00<00:00, 282.68it/s]


In [50]:
save_to_json(filename = 'CEOD_bal_acc_arr.json', data = CEOD_bal_acc_arr)
save_to_json(filename = 'CEOD_disp_imp_arr.json', data = CEOD_disp_imp_arr)
save_to_json(filename = 'CEOD_avg_odds_diff_arr.json', data = CEOD_avg_odds_diff_arr)
save_to_json(filename = 'CEOD_eq_opp_diff_arr.json', data = CEOD_eq_opp_diff_arr)
save_to_json(filename = 'CEOD_outcome_unfair_arr.json', data = CEOD_outcome_unfair_arr)
save_to_json(filename = 'CEOD_acc_equal_arr.json', data = CEOD_acc_equal_arr)

In [47]:
CEOD_bal_acc = np.interp(best_class_thresh, class_thresh_arr, CEOD_bal_acc_arr)
CEOD_acc_equal = np.interp(best_class_thresh, class_thresh_arr, CEOD_acc_equal_arr)
CEOD_disp_imp = np.interp(best_class_thresh, class_thresh_arr, CEOD_disp_imp_arr)
CEOD_out_unf = np.interp(best_class_thresh, class_thresh_arr, CEOD_outcome_unfair_arr)
CEOD_avg_odds = np.interp(best_class_thresh, class_thresh_arr, CEOD_avg_odds_diff_arr)
CEOD_eq_odds = np.interp(best_class_thresh, class_thresh_arr, CEOD_eq_opp_diff_arr)

In [48]:
CEOD_results = {'Metric': ['Balanced Accuracy', 'Accuracy Equality', 'Disparate Impact', 'Outcome Unfairess', 'Average Odds Difference', 'Equal Opportunity Difference'],
    'Calibrated Equal Odds': [CEOD_bal_acc, CEOD_acc_equal, CEOD_disp_imp, CEOD_out_unf, CEOD_avg_odds, CEOD_eq_odds]}
CEOD_results_df = pd.DataFrame(CEOD_results)
CEOD_results_df

,Metric,Calibrated Equal Odds
0,Balanced Accuracy,0.674342
1,Accuracy Equality,0.044767
2,Disparate Impact,0.591625
3,Outcome Unfairess,-0.191008
4,Average Odds Difference,-0.252650
5,Equal Opportunity Difference,-0.230266


In [49]:
df_to_json(filename = 'CEOD_results.json', df = CEOD_results_df)